# Doing a Natural Language Processing Analysis

In [2]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tensorflow as tf
import pandas as pd
import random
import numpy as np
import torch as torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from pysentimiento import create_analyzer

df = pd.read_csv('../data/reddit.csv')
df = df.fillna("")
df.head()

,bikes,cars,walk
0,Might want to repost on r/bikedc for a differe...,I probably wouldn't be living in the area if i...,I’ve had different favorites as I’ve aged. Whe...
1,Honestly the city as a whole has become very b...,I didn't even bring my car when I moved to the...,Mt. Pleasant. You go there/live there and you ...
2,"If you work downtown, there’s a protected bike...",I don't own a car. If not for the Metro (and M...,I'm surprised no one has said Capitol Hill
3,[Here’s a bike map from DDOT](https://ddot.dc....,My bike is a bigger reason my car is mostly a ...,Mt. Pleasant.\n\n&#x200B;\n\nI always said if ...
4,Are you planning to bike to work? For utility ...,"For me, it's not so much that I'm driving less...",I love Adams Morgan. There’s something for eve...


Adding a function to clean string

In [3]:
def remove_nonletters(strings):
    pattern = re.compile(r"[^a-zA-Z\s]|[\n]+")

    cleaned_strings = [(pattern.sub("", i)).lower() for i in strings]
    return cleaned_strings

In [4]:
for col in df.columns:
    col_clean = str(col)+"_clean"
    df[col_clean] = remove_nonletters(df[col])

df.head()

,bikes,cars,walk,bikes_clean,cars_clean,walk_clean
0,Might want to repost on r/bikedc for a differe...,I probably wouldn't be living in the area if i...,I’ve had different favorites as I’ve aged. Whe...,might want to repost on rbikedc for a differen...,i probably wouldnt be living in the area if it...,ive had different favorites as ive aged when i...
1,Honestly the city as a whole has become very b...,I didn't even bring my car when I moved to the...,Mt. Pleasant. You go there/live there and you ...,honestly the city as a whole has become very b...,i didnt even bring my car when i moved to the ...,mt pleasant you go therelive there and you go ...
2,"If you work downtown, there’s a protected bike...",I don't own a car. If not for the Metro (and M...,I'm surprised no one has said Capitol Hill,if you work downtown theres a protected bike l...,i dont own a car if not for the metro and metr...,im surprised no one has said capitol hill
3,[Here’s a bike map from DDOT](https://ddot.dc....,My bike is a bigger reason my car is mostly a ...,Mt. Pleasant.\n\n&#x200B;\n\nI always said if ...,heres a bike map from ddothttpsddotdcgovpagebi...,my bike is a bigger reason my car is mostly a ...,mt pleasantxbi always said if i could buy prop...
4,Are you planning to bike to work? For utility ...,"For me, it's not so much that I'm driving less...",I love Adams Morgan. There’s something for eve...,are you planning to bike to work for utility e...,for me its not so much that im driving less bu...,i love adams morgan theres something for every...


## obtain top words for wordcloud vis

In [5]:
#cleaning stopwords

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

s_words=list(stopwords.words('english'))
extra_s_words = ["the","to","a","and","of","i","is","you","it","i","is","ive","youve","im","dc","youre","dont","would","also"]
for i in extra_s_words:
     s_words.append(i)

x = ["bikes_clean",
     "cars_clean",
     "walk_clean"]

words = {}

for col in x:
     tokens = [word_tokenize(i) for i in df[col]]
     words[col] = " ".join([word for sentence in tokens for word in sentence if word not in s_words])
words

{'bikes_clean': 'might want repost rbikedc different side opinions honestly city whole become bike friendly theres many protected bike routes quadrants trails mbt metropolitan branch trail art anacostia riverwalk trail cct capital crescent trail rcpt rock creek park trail co canal act major paved arteries protected lanes th st sw th st swth st nw etc mvt custis wod virginia many many isnt get way nw across anacostia bike infra drops even getting better year yeardefinitely check follow rbikedc regularly give route recommendations trail conditions utilitarian riders work downtown theres protected bike lane th st euclid st neighborhoods like logan circle u st dupont east even columbia heights could work west end could work since theres protected eastwest route l sts heres bike map ddothttpsddotdcgovpagebicyclemaps heres list mapsresources wabahttpswabaorgresourcesmapsanddocuments planning bike work utility errands etc exercisemight help narrow woodley park hit rock creek park protected bi

In [6]:
fdists= {}

for key in words.keys():
    tokenized_words = nltk.tokenize.word_tokenize(words[key])
    fdist = FreqDist(tokenized_words)
    fdists[key] = fdist
fdists

{'bikes_clean': FreqDist({'bike': 130, 'get': 58, 'one': 41, 'lanes': 40, 'good': 40, 'trail': 38, 'st': 34, 'lane': 34, 'park': 32, 'like': 30, ...}),
 'cars_clean': FreqDist({'metro': 94, 'car': 90, 'traffic': 78, 'speed': 75, 'people': 75, 'drive': 62, 'like': 55, 'get': 54, 'city': 46, 'driving': 45, ...}),
 'walk_clean': FreqDist({'people': 582, 'city': 532, 'like': 468, 'car': 330, 'get': 262, 'live': 250, 'one': 218, 'much': 206, 'think': 203, 'philly': 192, ...})}

In [7]:
words_freq = []
for key_n in fdists.keys():
    fdist = (dict(fdists[key_n].most_common(30)))
    result =""
    total =  sum(fdist.values())
    for key, value in fdist.items():
        n = (value*100) // total
        result+= (key+" ")*n
    words_freq.append(result)
words_freq

['bike bike bike bike bike bike bike bike bike bike bike bike bike get get get get get get one one one one lanes lanes lanes lanes good good good good trail trail trail trail st st st lane lane lane park park park like like like street street street city city mbt mbt bikes bikes many many ride ride trails trails way way pretty pretty people people th th go go rock rock creek creek nw nw map map take take lock lock want want much much ',
 'metro metro metro metro metro metro metro car car car car car car traffic traffic traffic traffic traffic speed speed speed speed speed people people people people people drive drive drive drive like like like like get get get get city city city driving driving driving time time time one one think think go go tickets tickets cameras cameras live live cars cars bike bike need need even even use use drivers drivers every every day day work work take take camera camera make make limit limit ',
 'people people people people people people people people peo

In [8]:
df.to_csv("../data/cars.txt",columns=['cars_clean'],index=False)
df.to_csv("../data/bikes.txt",columns=['bikes_clean'],index=False)
df.to_csv("../data/walk.txt",columns=['walk_clean'],index=False)

## Sentiment Analysis

In [9]:
analyzer = create_analyzer(task="sentiment", lang="es")

In [ ]:
df = pd.read_csv("../data/walk.txt")
df = df.dropna
sentiment = [analyzer.predict(i) for i in df["walk_clean"]]
sent_label = [i.output for i in sentiment]
sent_prob = [i.probas for i in sentiment]
df_probs = pd.DataFrame(sent_prob)
df_probs["label"] = sent_label
df_probs["walk"] = df["walk_clean"]
df_probs.to_csv("../data/walk_sent.csv")

In [12]:
b_sent = pd.read_csv("../data/bikes_sent.csv")
c_sent = pd.read_csv("../data/cars_sent.csv")
w_sent = pd.read_csv("../data/walk_sent.csv")


In [13]:
# drop index
b_sent = b_sent.drop(columns=['Unnamed: 0'])
c_sent = c_sent.drop(columns=['Unnamed: 0'])
w_sent = w_sent.drop(columns=['Unnamed: 0'])

# add category
b_sent["category"] = "bike"
c_sent["category"] = "car"
w_sent["category"] = "walk"

b_sent = b_sent.rename(columns={"bikes": "text"})
c_sent = c_sent.rename(columns={"cars": "text"})
w_sent = w_sent.rename(columns={"walk": "text"})

print("bikes shape: ",b_sent.shape)
print("cars shape: ",c_sent.shape)
print("walk shape: ",w_sent.shape)

bikes shape:  (268, 6)
cars shape:  (389, 6)
walk shape:  (1859, 6)


In [14]:
df_sent = b_sent.append([c_sent,w_sent], ignore_index = True)
df_sent.shape

C:\Users\valer\AppData\Local\Temp\ipykernel_972\1270264562.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



(2516, 6)

In [15]:
df_sent.head()

,NEG,NEU,POS,label,text,category
0,0.005856,0.969829,0.024315,NEU,might want to repost on rbikedc for a differen...,bike
1,0.001424,0.098843,0.899733,POS,honestly the city as a whole has become very b...,bike
2,0.002620,0.888769,0.108611,NEU,if you work downtown theres a protected bike l...,bike
3,0.003321,0.954665,0.042015,NEU,heres a bike map from heres a list of mapsres...,bike
4,0.002263,0.715282,0.282455,NEU,woodley park can hit rock creek park which is ...,bike


In [16]:
x1 = df_sent.loc[df_sent["category"] == "walk", "POS"]
x0 = df_sent.loc[df_sent["category"] == "car", "POS"]
x2 = df_sent.loc[df_sent["category"] == "bike", "POS"]

trace1 = go.Histogram(
    x=x0, name='car',
    opacity=0.75
)
trace2 = go.Histogram(
    x=x1, name = 'walk',
    opacity=0.75
)
trace3 = go.Histogram(
    x=x2, name = 'bike',
    opacity=0.75
)

data = [trace2, trace3, trace1]
layout = go.Layout(barmode='overlay', title='Distribution of Sentiment POS of reviews based on Recommendation')
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='overlaid histogram')

Generate polarity value

In [17]:
polarity = []
for i in range(len(df_sent["label"])):
    if df_sent["POS"][i] > df_sent["NEG"][i]:
        polarity.append(df_sent["POS"][i])
    elif df_sent["POS"][i] < df_sent["NEG"][i]:
        polarity.append(df_sent["NEG"][i]*-1)

df_sent["polarity"] = polarity
df_sent.head(3)


,NEG,NEU,POS,label,text,category,polarity
0,0.005856,0.969829,0.024315,NEU,might want to repost on rbikedc for a differen...,bike,0.024315
1,0.001424,0.098843,0.899733,POS,honestly the city as a whole has become very b...,bike,0.899733
2,0.002620,0.888769,0.108611,NEU,if you work downtown theres a protected bike l...,bike,0.108611


In [26]:
trace1 = go.Scatter(
    x=df_sent['polarity'], y=df_sent['category'], mode='markers', name='points',
    marker=dict(color='rgb(102,0,0)', size=2, opacity=0.4)
)
trace2 = go.Histogram2dContour(
    x=df_sent['polarity'], y=df_sent['category'], name='density', ncontours=30,
    colorscale='Hot', reversescale=True, showscale=False,
    hovertemplate='<br>Polarity: %{x}<br>Category: %{y}<br>Comments: %{z}'
)
trace3 = go.Histogram(
    x=df_sent['polarity'], name='Polarity',
    marker=dict(color='rgb(214, 163, 32)'),
    yaxis='y2',
    hovertemplate='<br>%{x}<br>'
)
trace4 = go.Histogram(
    y=df_sent['category'], name='Category', marker=dict(color='rgba(148, 46, 46, 0.784)'),
    xaxis='x2'
)
data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    showlegend=False,
    autosize=False,
    width=800,
    height=750,
    xaxis=dict(
        domain=[0, 0.85],
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        domain=[0, 0.85],
        showgrid=False,
        zeroline=False
    ),
    margin=dict(
        t=50
    ),
    hovermode='closest',
    bargap=0,
    xaxis2=dict(
        domain=[0.85, 1],
        showgrid=False,
        zeroline=False
    ),
    yaxis2=dict(
        domain=[0.85, 1],
        showgrid=False,
        zeroline=False
    )
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(yaxis_title="Category",xaxis_title="Polarity", title="Sentiment polarity of posts by category") 

iplot(fig, filename='2dhistogram-2d-density-plot-subplots')

In [24]:
y0 = df_sent.loc[df_sent['category'] == 'bike']['polarity']
y1 = df_sent.loc[df_sent['category'] == 'walk']['polarity']
y2 = df_sent.loc[df_sent['category'] == 'car']['polarity']

trace0 = go.Box(
    y=y0,
    name = 'bike',
    marker = dict(
        color = 'rgba(148, 46, 46, 0.784)',
    ),
    notched=True
)
trace1 = go.Box(
    y=y1,
    name = 'walk',
    marker = dict(
        color = 'rgb(153, 91, 40)',
    ),
    notched=True
)
trace2 = go.Box(
    y=y2,
    name = 'car',
    marker = dict(
        color = 'rgb(214, 163, 32)',
    ),
    notched=True
)

data = [trace0, trace1, trace2]
layout = go.Layout(
    title = "Sentiment polarity Boxplot of category"
)

fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = "Sentiment polarity Boxplot of category")